Disclaimer : This is the R Markdown File for the Task C for the final project.

In [ ]:

rm(list = ls())
getwd()
library("dplyr")
library(ggplot2)
library(ggthemes)
library(ggpubr)
setwd("D:\\BSE\\BSE Material\\sem 2\\Data Vis\\Project")

C1. Examine the house prices for 2015. How do these change over time? Do property prices seem to increase or decrease throughout the year?

Ans: The questions wants us to see the see if month plays a factor in price of properties in the UK. To visualize this trend we need to filter years belonging to year 2015. To help understand if there is a relationship between time of the year and price we further break down our dataset and add a subcategory of Quarters based on months. Like any financial document, we break the months in Quarterst starting from 1. The Quarters start from  the month of January. As there can be externalities in the data which might not allows us to visualize the results we create a variable called upper limit which takes the vlaues till the second standard deviation of the price distribution. 

In [ ]:
pp_data <- read.csv("ppdata_lite.csv")
Monthly <-  pp_data%>%
  mutate(year= format(as.Date(date_of_transfer, format = "%Y-%m-%d"),"%Y"),
    month = format(as.Date(date_of_transfer, format = "%Y-%m-%d" ),"%m"))

  
Monthly <- Monthly%>%
  mutate(quarters = case_when(month %in% sprintf("%02d", 1:3)~"Q1",
                               month %in% sprintf("%02d", 4:6)~"Q2",
                               month %in% sprintf("%02d", 7:9)~"Q3",
                               month %in% seq(10,12,1)~"Q4"
                               ))
upper_limit = mean(Monthly$price) + 2 * sd(Monthly$price)

We filter the data using the upper limit set. 

In [ ]:
plot_data <- Monthly %>% 
  filter(year == 2015)%>%
  arrange(month)%>%
  filter(!price>upper_limit)%>%
  group_by(month,quarters)%>%
  summarize(mean_price = mean(price),
            median_price = median(price),.groups= "keep")
head(plot_data)

We then plot the trend data using both mean and median price values. This would allow us to get a better idea. A difference in mean and median would further clarify if there is any skewness in our data. 

In [ ]:

month_plot_mean <- ggplot(plot_data,aes(x= month, y = mean_price)) + 
  geom_point(aes(colour= quarters))+geom_path(aes(group = 1,color = quarters))+xlab("Months")+ylab("Prices")+
  ggtitle("Mean Price Change in 2015")+scale_colour_discrete("Quarters")+theme_pander()


month_plot_median <- ggplot(plot_data,aes(x= month, y = median_price)) + 
  geom_point(aes(colour= quarters))+geom_path(aes(group = 1,color = quarters))+xlab("Months")+ylab("Prices")+
  ggtitle("Median Price Change in 2015")+scale_colour_discrete("Quarters")+theme_pander()
  

figure <- ggarrange(month_plot_median,
                    month_plot_mean,
                    ncol = 1, nrow = 2)
figure


Both Graphs indicate the same trend. Price are the lowest in the begging on the year with an drastic rise by the Q3(July-Sept). With the peak in August the prices reduce to the level which we could see at the beginning of July. This does indicate a trend in 2015. To add more weight to the argument that this trend is relevant it is good to observe the past trends for the same relationship between quarters.

In [ ]:

plot_data_2 <- Monthly %>% 
  arrange(month)%>%
  filter(!price>upper_limit)%>%
  group_by(year,quarters)%>%
  summarize(mean_price = mean(price),
            median_price = median(price),.groups= "keep")
head(plot_data_2)

Quarter_plot_median <- ggplot(plot_data_2,aes(y= median_price, x = year))+
  geom_line(aes(colour = quarters, group = quarters))+
  scale_x_discrete(guide = guide_axis(angle = 45))+ylab("Prices")+
  ggtitle("Quarter trends")+scale_colour_discrete("Quarters")+
  scale_y_continuous(breaks = round(seq(min(plot_data_2$median_price), max(plot_data_2$median_price), by = 25000),1))+
  theme_pander()

Quarter_plot_median 

C2. Is there a significant relationship between the price of a property and the time of year it is sold? Does this vary with type of property?

Ans : To understand if this relationship exist over the years we need to continue to use our main dataset with some changes. The dataset has 5 property types, Detached, Semi-Detached, Terraced ,Flats, Others. We now need to see how this relationship persesits over the years as well as to get a better idea we look at the data of 2015 as well. 

Lets see the data between 2007-2016 for the different properties and how prices change with property types. We filter the data from 2007 to visualize other properties. 

In [ ]:
plot_data_3 <- Monthly %>% 
  arrange(month)%>%
  filter(!price>upper_limit,
         year > 2006)%>%
  mutate(property_type = case_when(property_type == "D" ~ "Detached",
                                   property_type == "S" ~ "Semi-Detached",
                                   property_type == "F" ~ "Flats",
                                   property_type == "T" ~ "Terraced",
                                   property_type == "O" ~ "Other"))%>%
  group_by(year,property_type)%>%
  summarize(mean_price = mean(price),
            median_price = median(price),.groups= "keep")
head(plot_data_3)

Property_plot_median <- ggplot(plot_data_3,aes(y= median_price, x = year))+
  geom_line(aes(colour = property_type, group = property_type))+
  scale_x_discrete(guide = guide_axis(angle = 45))+ylab("Prices")+
  ggtitle("Property Type trends")+scale_colour_discrete("Property Types")+
  scale_y_continuous(breaks = round(seq(min(plot_data_3$median_price), max(plot_data_3$median_price), by = 25000),1))
Property_plot_median


As Other property seems to be a noise we further plot the time series data for the remaining properties and visualize the trend. 

In [ ]:

plot_data_3b <- Monthly %>% 
  arrange(month)%>%
  filter(!price>upper_limit,
         !property_type == 'O')%>%
  mutate(property_type = case_when(property_type == "D" ~ "Detached",
                                   property_type == "S" ~ "Semi-Detached",
                                   property_type == "F" ~ "Flats",
                                   property_type == "T" ~ "Terraced"))%>%
  group_by(year,property_type)%>%
  summarize(mean_price = mean(price),
            median_price = median(price),.groups= "keep")

head(plot_data_3b)

Property_plot_median_b <- ggplot(plot_data_3b,aes(y= median_price, x = year))+
  geom_line(aes(colour = property_type, group = property_type))+
  scale_x_discrete(guide = guide_axis(angle = 90))+ylab("Prices")+
  ggtitle("Property Type trends")+scale_colour_discrete("Property Types")+
  scale_y_continuous(breaks = round(seq(min(plot_data_3b$median_price), max(plot_data_3b$median_price), by = 25000),1))

Property_plot_median_b 

We find that Detached properties are the costliest. Semi-Detached and Flats are valued simillarly with terrace properties being cheapest option. 

Combining our previous trend data that represented how different quarters (time of the year) effects price of property we can further try and see if how the prices of properties are effected througout the year. We again go select the year as 2015 to maining uniformity in our observation. 

In [ ]:

plot_data4 <- Monthly %>% 
  filter(year == 2015)%>%
  mutate(property_type = case_when(property_type == "D" ~ "Detached",
                                   property_type == "S" ~ "Semi-Detached",
                                   property_type == "F" ~ "Flats",
                                   property_type == "T" ~ "Terraced",
                                   property_type == "O" ~ "Other"))%>%
  arrange(month)%>%
  filter(!price>upper_limit)%>%
  group_by(month,quarters,property_type)%>%
  summarize(mean_price = mean(price),
            median_price = median(price),.groups= "keep")
head(plot_data4)

Property_quarters <- ggplot(plot_data4,aes(x= month, y = median_price)) + 
  geom_point(aes(colour= quarters))+geom_path(aes(group = 1,color = quarters))+xlab("Months")+ylab("Prices")+
  ggtitle("Property Price Change in Quarters of 2015")+scale_colour_discrete("Quarters")+theme_pander()+
  scale_x_discrete(labels=seq(1,12,1))+facet_wrap(~property_type)

Property_quarters


We can observe that flats are pretty stationary as compared to other properties with the most variations being in the properties that are not categorized. Moreover, Detached,Semi-Detached and Terraced find similar moments.
